In [1]:
# --- IMPORT LIBS ---
import os
#my_dir = "/home/oddc/crypto_robot_live/"
my_dir = "/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/"
lib_file_path = os.path.join(my_dir, 'utilities')
print(lib_file_path)
import sys
sys.path.append(my_dir + 'utilities')
import ta
from perp_bybit import *
from datetime import datetime
import json
import uuid
import logging

/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/utilities


In [2]:
# --- CONFIGURE LOGGER --
log_file_path = os.path.join(my_dir, 'my_code', 'log_file.txt')
print(log_file_path)
logging.basicConfig(filename = log_file_path, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

showLog = True

def log_me(message):
    if showLog:
        print(message)
    else:
        logging.info(message)

/Users/olivierdedecker/Documents/00_Dev/Python/Crypto_Robot_live/my_code/log_file.txt


In [3]:
# --- LAUNCH ---
showLog = True                      #
now = datetime.now()
current_time = now.strftime("%d/%m/%Y %H:%M:%S")
current_time_python = now.timestamp()
log_me("")
log_me(f"--- Start Execution Time : {current_time} ---")


--- Start Execution Time : 03/07/2023 21:59:16 ---


In [4]:
# --- PARAMETERS & VARIABLES ---
# -- Account --
secret_file_path = os.path.join(my_dir, 'live_tools', 'secret.json')
f = open(secret_file_path)
secret = json.load(f)
f.close()

exchange_name = 'bybit'
account_to_select = 'testnet_account'
production = True

# -- Coins & timeframe --
timeframe = '1h'
pair = "VET/USDT:USDT"
leverage = 1
log_me(f"--- {pair} {timeframe} Leverage x {leverage} ---")

# -- Indicator variable --
ema_shifts = [0.05, 0.1, 0.15]
ema_period = 5

# -- Rules --
#max_coins_in_position = 1           # max number of coin positions
nLevel = len(ema_shifts)            # max number of open positions per coin
position_type = ["long", "short"]
open_position_asap = True
close_position_with_indicator = False

--- VET/USDT:USDT 1h Leverage x 1 ---


In [5]:
# --- FUNCTIONS ---
def open_long(row):
    if ('long' in position_type) and open_position_asap:
        return True
    else:
        return False

def close_long(row):
    if close_position_with_indicator:
        # some code
        return True
    else:
        return False

def open_short(row):
    if ('short' in position_type) and open_position_asap:
        return True
    else:
        return False

def close_short(row):
    if close_position_with_indicator:
        # some code
        return True
    else:
        return False

In [6]:
# --- INITIALIZE EXCHANGE & GET BALANCE ---
# connect exchange
bybit = PerpBybit(
    apiKey=secret[account_to_select]["apiKey"],
    secret=secret[account_to_select]["secret"],
    default_type='swap',
    is_real=secret[account_to_select]["is_real"]
)

# get portfolio balance data from exchange
usdt_equity = float(bybit.get_usdt_equity())
usdt_available_balance = float(bybit.get_usdt_available_balance())
log_me(f'available usdt balance: {usdt_available_balance:.2f} $')

# get balance, position and order data
usd_balance = float(bybit.get_usdt_equity())
log_me(f"USD balance: {usd_balance:.2f} $")

positions_data = bybit.get_open_position()
position_list = [
    {"side": d["side"], "size": float(d["contracts"]) * float(d["contractSize"]), "market_price":d["markPrice"], "usd_size": float(d["contracts"]) * float(d["contractSize"]) * float(d["markPrice"]), "open_price": d["entryPrice"]}
    for d in positions_data if d["symbol"] == pair]
df_position = pd.DataFrame(position_list)
log_me('Positions')
log_me(df_position)

orders_list = []
for order in bybit.get_open_orders():
    orders_list.append(order["info"])
df_orders = pd.DataFrame(orders_list)
if df_orders.empty == False:
    df_orders["price"] = pd.to_numeric(df_orders["price"])
    df_orders["qty"] = pd.to_numeric(df_orders["qty"])
log_me('Open orders')
log_me(df_orders)

# Get data
df = bybit.get_more_last_historical_async(pair, timeframe, 1000)

hey .. set sandbox mode ON
available usdt balance: 653.14 $
USD balance: 734.45 $
Positions
Empty DataFrame
Columns: []
Index: []
Open orders
                                orderId  \
0  04edb31e-76ea-4d35-b0f5-16c59caf3c49   
1  9a3f30c6-0869-43c0-a196-eaa2793dcea0   
2  6e9bcea8-87da-4fc8-a09c-b87b36f2a212   
3  1fd8eb57-0c9e-4c3c-ba70-43a2509cbb17   
4  706c58ae-38ea-4705-a9cf-abf3dc5d0cf5   
5  786cdd22-86da-4434-8a3f-1e6b064ea8d7   

                                     orderLinkId blockTradeId   symbol  \
0  ema_low_1#c7667827-69b2-4197-9a51-073d8c9fb6a               VETUSDT   
1  ema_low_2#c7667827-69b2-4197-9a51-073d8c9fb6a               VETUSDT   
2  ema_low_3#c7667827-69b2-4197-9a51-073d8c9fb6a               VETUSDT   
3  ema_high_1#c7667827-69b2-4197-9a51-073d8c9fb6               VETUSDT   
4  ema_high_2#c7667827-69b2-4197-9a51-073d8c9fb6               VETUSDT   
5  ema_high_3#c7667827-69b2-4197-9a51-073d8c9fb6               VETUSDT   

     price    qty  side isLeverage po

In [7]:

# --- POPULATE INDICATORS ---
sell_ema_values={}
buy_ema_values={}
df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)

df['ema_base'] = ta.trend.ema_indicator(close=df['close'], window=ema_period)
for i, shift in enumerate(ema_shifts, start=1):
    df[f'ema_high_{i}'] = df['ema_base'] * (1 + shift)
    df[f'ema_low_{i}'] = df['ema_base'] * (1 - shift)
    sell_ema_values[f'ema_high_{i}'] = bybit.convert_price_to_precision(pair, df.iloc[-1][f'ema_high_{i}'])
    buy_ema_values[f'ema_low_{i}'] = bybit.convert_price_to_precision(pair, df.iloc[-1][f'ema_low_{i}'])


In [8]:
# --- CANCEL OPEN UNFILLED ORDERS ---
cancelled_longs = []
cancelled_shorts = []

for order in orders_list:
    if order['orderLinkId'] != '' and order['side']=='Buy':
        enveloppe_id = order['orderLinkId'].split("#")[0]
        cancelled_longs.append(enveloppe_id)
        order_id = order['orderId']
        order_symbol = order['symbol']
        log_me(f'Cancel buy order {order_id}')
        if production:
            bybit.cancel_order(order_id = order_id, symbol=order_symbol)
    if order['orderLinkId'] != '' and order['side']=='Sell':
        enveloppe_id = order['orderLinkId'].split("#")[0]
        cancelled_shorts.append(enveloppe_id)
        order_id = order['orderId']
        order_symbol = order['symbol']
        log_me(f'Cancel sell order {order_id}')
        if production:
            bybit.cancel_order(order_id = order_id, symbol=order_symbol)

log_me(f'Cancelled {len(cancelled_longs)} longs and {len(cancelled_shorts)} shorts')

Cancel buy order 04edb31e-76ea-4d35-b0f5-16c59caf3c49
Cancel buy order 9a3f30c6-0869-43c0-a196-eaa2793dcea0
Cancel buy order 6e9bcea8-87da-4fc8-a09c-b87b36f2a212
Cancel sell order 1fd8eb57-0c9e-4c3c-ba70-43a2509cbb17
Cancel sell order 706c58ae-38ea-4705-a9cf-abf3dc5d0cf5
Cancel sell order 786cdd22-86da-4434-8a3f-1e6b064ea8d7
Cancelled 3 longs and 3 shorts


In [9]:
df_orders

,orderId,orderLinkId,blockTradeId,symbol,price,qty,side,isLeverage,positionIdx,orderStatus,...,smpType,smpGroup,smpOrderId,tpslMode,tpLimitPrice,slLimitPrice,placeType,createdTime,updatedTime,nextPageCursor
0,04edb31e-76ea-4d35-b0f5-16c59caf3c49,ema_low_1#c7667827-69b2-4197-9a51-073d8c9fb6a,,VETUSDT,0.01955,11826,Buy,,0,New,...,None,0,,,,,,1688412639783,1688412639785,NaN
1,9a3f30c6-0869-43c0-a196-eaa2793dcea0,ema_low_2#c7667827-69b2-4197-9a51-073d8c9fb6a,,VETUSDT,0.01850,11826,Buy,,0,New,...,None,0,,,,,,1688412640299,1688412640302,NaN
2,6e9bcea8-87da-4fc8-a09c-b87b36f2a212,ema_low_3#c7667827-69b2-4197-9a51-073d8c9fb6a,,VETUSDT,0.01750,11826,Buy,,0,New,...,None,0,,,,,,1688412640503,1688412640506,NaN
3,1fd8eb57-0c9e-4c3c-ba70-43a2509cbb17,ema_high_1#c7667827-69b2-4197-9a51-073d8c9fb6,,VETUSDT,0.02160,11826,Sell,,0,New,...,None,0,,,,,,1688412640708,1688412640711,NaN
4,706c58ae-38ea-4705-a9cf-abf3dc5d0cf5,ema_high_2#c7667827-69b2-4197-9a51-073d8c9fb6,,VETUSDT,0.02265,11826,Sell,,0,New,...,None,0,,,,,,1688412641228,1688412641230,NaN
5,786cdd22-86da-4434-8a3f-1e6b064ea8d7,ema_high_3#c7667827-69b2-4197-9a51-073d8c9fb6,,VETUSDT,0.02365,11826,Sell,,0,New,...,None,0,,,,,,1688412641419,1688412641421,page_args%3D04edb31e-76ea-4d35-b0f5-16c59caf3c...


In [10]:
# --- CREATE AND MODIFY ORDERS ---
# deal entry logic
row = df.iloc[-2]
unique_id = f"#{uuid.uuid4()}"

# Determine order size
#available_positions = 1
market_price = float(df.iloc[-1]['close'])
usdt_position_size = usdt_available_balance #/ available_positions
usdt_order_size = (usdt_position_size*leverage) / nLevel
log_me(f'usdt order size for new orders: {usdt_order_size} using {leverage}x leverage')
coin_order_size = usdt_order_size / market_price
rounded_coin_order_size = float(bybit.convert_amount_to_precision(pair, coin_order_size))
log_me(f'coin order size for new orders: {coin_order_size}')
log_me(f'rounded coin order size for new orders: {rounded_coin_order_size}')

# Adjust TP of open position
if len(positions_data) > 0:
    log_me("Active position")
    for order in orders_list:
        if order['orderLinkId']=='':
            order_size = order['qty']
            order_tp = bybit.convert_price_to_precision(pair, df.iloc[-1]['ema_base'])
            log_me(f"Modify position TP: {order_size} {pair} at the price of {order_tp}$")
            if production:
                order = bybit.edit_order(
                    id=order['orderId'],
                    symbol=order['symbol'],
                    type=order['orderType'],
                    side=order['side'],
                    amount=order['qty'],
                    price=None,
                    params={
                        'triggerPrice': bybit.convert_price_to_precision(pair, df.iloc[-1]['ema_base'])
                    }
                )

# Create new limit orders for remaining slots
if open_long(row) and "long" in position_type:
    for ema, ema_value in buy_ema_values.items():
        if (ema in cancelled_longs) or df_orders.empty or len(positions_data)==0:
            log_me(f"Place {ema} Long Limit Order: {rounded_coin_order_size} {pair} at the price of {ema_value}$")
            if production:
                order = bybit.place_limit_order(
                    symbol=pair,
                    side='buy',
                    amount=rounded_coin_order_size,
                    limit=ema_value,
                    sl=None,
                    tp=bybit.convert_price_to_precision(pair, df.iloc[-1]['ema_base']),
                    reduce=False,
                    orderLinkId=f'{ema}{unique_id}'
                )

if open_short(row) and "short" in position_type:
    for ema, ema_value in sell_ema_values.items():
        if (ema in cancelled_shorts) or df_orders.empty or len(positions_data)==0:
            log_me(f"Place {ema} Short Limit Order: {rounded_coin_order_size} {pair} at the price of {ema_value}$")
            if production:
                order = bybit.place_limit_order(
                    symbol=pair,
                    side='sell',
                    amount=rounded_coin_order_size,
                    limit=ema_value,
                    sl=None,
                    tp=bybit.convert_price_to_precision(pair, df.iloc[-1]['ema_base']),
                    reduce=False,
                    orderLinkId=f'{ema}{unique_id}'
                )


usdt order size for new orders: 217.71401982666666 using 1x leverage
coin order size for new orders: 10568.641739158575
rounded coin order size for new orders: 10568.0
Place ema_low_1 Long Limit Order: 10568.0 VET/USDT:USDT at the price of 0.0195$
Place ema_low_2 Long Limit Order: 10568.0 VET/USDT:USDT at the price of 0.0185$
Place ema_low_3 Long Limit Order: 10568.0 VET/USDT:USDT at the price of 0.01745$
Place ema_high_1 Short Limit Order: 10568.0 VET/USDT:USDT at the price of 0.02155$
Place ema_high_2 Short Limit Order: 10568.0 VET/USDT:USDT at the price of 0.0226$
Place ema_high_3 Short Limit Order: 10568.0 VET/USDT:USDT at the price of 0.02365$


In [11]:
# --- CLOSE ---
now = datetime.now()
current_time = now.strftime("%d/%m/%Y %H:%M:%S")
log_me(f"--- End Execution Time: {current_time} ---")
log_me("")

--- End Execution Time: 03/07/2023 22:00:16 ---

